In [51]:
!nvidia-smi

Wed Nov 29 03:05:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    34W /  70W |  14865MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Imports

In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.4 MB/s eta 0:00:00


In [ ]:
!pip install datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=f30b63c5fb550e82cd34238f5ed9f08c7856cd0378047e664d98e80543c1b451
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
data_drive = "drive/MyDrive/ebay_data_2023"
os.listdir(data_drive)

['Train_Tagged_Titles.tsv',
 'Listing_Titles.tsv',
 'train_data_ffilled.csv',
 'results_1119.tsv',
 'results_1120_Itagfixed.tsv',
 'results_1120_NoTagObscureRemoved.tsv']

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install distance
import distance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16256 sha256=3ba1888a6cf285aa288717feaab1e1bcb9e2765e93a9141a344a5e28a63d558b
  Stored in directory: /root/.cache/pip/wheels/e8/bb/de/f71bf63559ea9a921059a5405806f7ff6ed612a9231c4a9309
Successfully built distance


In [ ]:
import pandas as pd
import numpy as np
import csv

In [ ]:
from transformers import AutoTokenizer
import datasets
from transformers import DataCollatorForTokenClassification
import evaluate
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from torch.optim import AdamW
from transformers import get_scheduler
from tensorflow.python.client import device_lib

## Preprocess Data

In [ ]:
train_df = pd.read_csv(data_drive + "/Train_Tagged_Titles.tsv", sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)
old_tag = train_df.Tag.copy()
train_df

,Record Number,Title,Token,Tag
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,NaN
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ
...,...,...,...,...
55178,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Sportschuhe,Produktart
55179,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Wanderschuh,NaN
55180,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Big,No Tag
55181,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Size,No Tag


In [ ]:
#quiz dataset
test_records = pd.read_csv(data_drive + "/Listing_Titles.tsv", sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE).loc[5000:29999,:].reset_index(drop=True)


In [ ]:
aspect_names = set(pd.unique(old_tag)) - set([np.nan])
aspect_names, len(aspect_names) # includes "No Tag", "Obscure"

({'Abteilung',
  'Aktivität',
  'Akzente',
  'Anlass',
  'Besonderheiten',
  'Charakter',
  'Charakter Familie',
  'Dämpfungsgrad',
  'EU-Schuhgröße',
  'Erscheinungsjahr',
  'Farbe',
  'Futtermaterial',
  'Gewebeart',
  'Herstellernummer',
  'Herstellungsland und -region',
  'Innensohlenmaterial',
  'Jahreszeit',
  'Laufsohlenmaterial',
  'Marke',
  'Maßeinheit',
  'Modell',
  'Muster',
  'No Tag',
  'Obermaterial',
  'Obscure',
  'Produktart',
  'Produktlinie',
  'Schuhschaft-Typ',
  'Schuhweite',
  'Stil',
  'Stollentyp',
  'Thema',
  'UK-Schuhgröße',
  'US-Schuhgröße',
  'Verschluss',
  'Zwischensohlen-Typ'},
 36)

In [ ]:
train_df.Tag = train_df.Tag.fillna("I-" + train_df.Tag.ffill()[train_df.Tag.isna()])
train_df.Tag.loc[train_df.Tag == "I-No Tag"] = "No Tag" # correct for the No Tag (there can be no interior)
train_df.Tag.loc[train_df.Tag == "I-Obscure"] = "Obscure" # correct for the No Tag (there can be no interior)

In [ ]:
# train_df.to_csv(data_drive + "/train_dfBI.csv")

In [ ]:
pd.unique(train_df.Tag), len(pd.unique(train_df.Tag)) # notice this does not encompass all 70 classes (36*2) - 2 (for No Tag, Obscure)
pass

In [ ]:
not_to_include = set(["No Tag", "Obscure"])
asp_arr = np.array(list(aspect_names - not_to_include))
asp_arr.sort()
interior_arr = ("I-"+ pd.Series(list(aspect_names - not_to_include))).sort_values().values
all_aspect_names = np.append(asp_arr, interior_arr)
all_aspect_names = np.append(all_aspect_names, list(not_to_include))
#all aspect names, all interior aspect names, Obscure, No Tag = 70 aspect names

In [ ]:
id2label = dict(zip(list(range(len(all_aspect_names))), all_aspect_names))
label2id = dict(zip(all_aspect_names, list(range(len(all_aspect_names)))))

In [ ]:
train_df["ner_tags"] = train_df.Tag.apply(lambda val: label2id[val])
train_df["Record Number"] = train_df["Record Number"].astype(int)

## Tokenize

The next step is to load a DistilBERT tokenizer to preprocess the `tokens` field:

In [ ]:
lowercased_tokenizer = AutoTokenizer.from_pretrained("numind/entity-recognition-general-sota-v1", do_lower_case=False, add_prefix_space=True)
lowercased_tokenizer.do_lower_case = True

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

As you saw in the example `tokens` field above, it looks like the input has already been tokenized. But the input actually hasn't been tokenized yet and you'll need to set `is_split_into_words=True` to tokenize the words into subwords. For example:

In [ ]:
tokenized_input = lowercased_tokenizer("Here is Wolfgang sentence".lower())
tokens = lowercased_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['<s>', 'Ġhere', 'Ġis', 'Ġwolf', 'gang', 'Ġsentence', '</s>']

In [ ]:
tokenized_input = lowercased_tokenizer("Here is Wolfgang sentence")
tokens = lowercased_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
[s.lower() for s in tokens]

['<s>', 'ġhere', 'ġis', 'ġwolfgang', 'ġsentence', '</s>']

In [ ]:
train_ds = datasets.Dataset.from_pandas(train_df)
train_ds

Dataset({
    features: ['Record Number', 'Title', 'Token', 'Tag', 'ner_tags'],
    num_rows: 55183
})

In [ ]:
example = train_df.loc[train_df["Record Number"] == 1, ["Record Number", "Token", "ner_tags"]].to_dict(orient="list")
example

{'Record Number': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'Token': ['Supreme',
  'Nike',
  'SB',
  'Dunk',
  'High',
  'By',
  'any',
  'Means',
  'Red',
  'US10',
  'EU44',
  'Supreme',
  'Box',
  'Logo',
  'Air',
  'Force'],
 'ner_tags': [20, 18, 24, 58, 25, 20, 54, 54, 10, 31, 8, 69, 69, 2, 24, 58]}

In [ ]:
datasets.Dataset.from_dict(example)["ner_tags"]

[20, 18, 24, 58, 25, 20, 54, 54, 10, 31, 8, 69, 69, 2, 24, 58]

In [ ]:
def combine(x):
  d = x.to_dict(orient="list")
  d["id"] = d["Record Number"][0]
  del d["Record Number"]
  del d["Title"]
  del d["Tag"]
  d["tokens"] = d.pop("Token")
  d["tokens"] = [s.lower() for s in d["tokens"]]
  return d["ner_tags"], d["tokens"]

In [ ]:
pre_ds = pd.DataFrame(train_df.groupby(["Record Number", "Title"]).apply(combine))

In [ ]:
pre_ds

,,0
Record Number,Title,
1,Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force,"([20, 18, 24, 58, 25, 20, 54, 54, 10, 31, 8, 6..."
2,New Balance 530 Männer und Frauen Laufschuhe mit Buchstaben N bequeme Laufschuhe,"([18, 52, 20, 0, 69, 0, 23, 69, 21, 55, 69, 23..."
3,♥ MICHAEL KORS Sneaker Gr 39 ♥,"([69, 18, 52, 27, 69, 8, 69], [♥, michael, kor..."
4,New Balance ML 574 EGO Turnschuhe grün,"([18, 52, 20, 54, 54, 23, 10], [new, balance, ..."
5,"Nike air jordan 9 og , space jam , hare , bugs bunny , EU 45 / US 11 NEU + RECHNUNG","([18, 24, 58, 20, 54, 69, 10, 20, 69, 10, 69, ..."
...,...,...
4996,Adidas Raf Simons Sneakers,"([18, 24, 58, 27], [adidas, raf, simons, sneak..."
4997,"Ludwig Reiter Andalusier , Nubuk navy Schuhe Damen Gr . 39 NEU","([18, 52, 20, 69, 22, 10, 23, 0, 69, 69, 8, 69..."
4998,Adidas Superstar Laceless / Adidas Originals / Adidas Sneaker / weiß / schwarz / FV3017,"([18, 24, 32, 69, 18, 24, 69, 18, 27, 69, 10, ..."


In [ ]:
pre_ds[['ner_tags', 'tokens']] = pd.DataFrame(pre_ds[0].tolist(), index=pre_ds.index)
pre_ds = pre_ds.reset_index().loc[:, ["Record Number", "tokens", "ner_tags"]].rename(columns={"Record Number":"id"})

In [ ]:
ebay_ds = datasets.Dataset.from_pandas(pre_ds)
ebay_ds

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 5000
})

However, this adds some special tokens `[CLS]` and `[SEP]` and the subword tokenization creates a mismatch between the input and labels. A single word corresponding to a single label may now be split into two subwords. You'll need to realign the tokens and labels by:

1. Mapping all tokens to their corresponding word with the [`word_ids`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding.word_ids) method.
2. Assigning the label `-100` to the special tokens `[CLS]` and `[SEP]` so they're ignored by the PyTorch loss function.
3. Only labeling the first token of a given word. Assign `-100` to other subtokens from the same word.

Here is how you can create a function to realign the tokens and labels, and truncate sequences to be no longer than DistilBERT's maximum input length:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = lowercased_tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [ ]:
tokenized_ebay = ebay_ds.map(tokenize_and_align_labels, batched=True)
tokenized_ebay

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=lowercased_tokenizer)

## Evaluate


In [ ]:
seqeval = evaluate.load("seqeval")

In [ ]:
label_list = list(label2id.keys())
len(label_list)

70

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    # predictions = predictions.logits #it is a tuple with size 1
    np_predictions = np.array(predictions)
    # print(np_predictions.shape)
    # print(predictions[0])
    # print(predictions[1])
    # print(labels)
    predictions = np.argmax(predictions[0], axis=2) #0 picks the first item in the returned outputs [logits, hidden_states]

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
batch_size = 16
sequence_length = 11
num_classes = 70
input_data = torch.randn((batch_size, sequence_length, num_classes))  # Adjust dimensions based on your actual input
label_data = torch.randint(0, num_classes, (batch_size, sequence_length))

In [ ]:
text = "Here is some text"

In [ ]:
# input_data = lowercased_tokenizer("Here is some text", return_tensors="pt")
# with torch.no_grad():
#   output = model(input_data.input_ids)

In [ ]:
# logits = output.logits

# # Assuming you have labels or targets for evaluation
# labels = torch.randint(0, 2, (1, len(lowercased_tokenizer.encode(text))))

# # Convert logits and labels to numpy arrays
# logits_np = logits.numpy()
# labels_np = labels.numpy()

# # Assuming you have your label_list defined
# # label_list = ["label_0", "label_1"]

# # Call your compute_metrics method
# metrics = compute_metrics((logits_np, labels_np))

# # Print or use the metrics as needed
# print(metrics)

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForTokenClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForTokenClassification) along with the number of expected labels, and the label mappings:

In [ ]:
tokenized_ebay.set_format("torch")

In [ ]:
tokenized_ebay_split = tokenized_ebay.train_test_split(0.2, shuffle=True)
tokenized_ebay_train = tokenized_ebay_split["train"]
tokenized_ebay_valid = tokenized_ebay_split["test"]

In [ ]:
train_dataloader = DataLoader(tokenized_ebay_train, shuffle=True, batch_size=8, collate_fn = data_collator) #8
eval_dataloader = DataLoader(tokenized_ebay_valid, batch_size=8) #8

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    # 'numind/entity-recognition-general-sota-v1', #english only
    'numind/generic-entity_recognition_NER-multilingual-v1', #multilingual
    output_hidden_states=True, num_labels=70, id2label=id2label, label2id=label2id
)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at numind/generic-entity_recognition_NER-multilingual-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12922533477401802094
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14410383360
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13576790900320288171
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

At this point, only three steps remain:

1. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the seqeval scores and save the training checkpoint.
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [ ]:
'''training_args = TrainingArguments(
    output_dir="roberta_ebay_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16, #16
    per_device_eval_batch_size=16, #16
    num_train_epochs=15,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ebay_train,
    eval_dataset=tokenized_ebay_valid,
    tokenizer=lowercased_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
'''

'training_args = TrainingArguments(\n    output_dir="roberta_ebay_model",\n    learning_rate=2e-5,\n    per_device_train_batch_size=16, #16\n    per_device_eval_batch_size=16, #16\n    num_train_epochs=15,\n    weight_decay=0.01,\n    evaluation_strategy="epoch",\n    save_strategy="epoch",\n    load_best_model_at_end=True,\n)\n\ntrainer = Trainer(\n    model=model,\n    args=training_args,\n    train_dataset=tokenized_ebay_train,\n    eval_dataset=tokenized_ebay_valid,\n    tokenizer=lowercased_tokenizer,\n    data_collator=data_collator,\n    compute_metrics=compute_metrics,\n)\n\ntrainer.train()\n'

In [ ]:
batch_sizes = [8]
learning_rates = [5e-6]
#function to obtain tag predictions and merge aspect values
def process_title(title, gpu=True):
    inputs = lowercased_tokenizer(title.lower(), return_tensors="pt")
    if gpu:
        cuda_in = inputs.input_ids.to("cuda")
    else:
        cuda_in = inputs.input_ids
    decoded = lowercased_tokenizer.convert_ids_to_tokens(cuda_in[0])

    model.eval()
    with torch.no_grad():
        logits = model(cuda_in).logits
    predictions = torch.argmax(logits, dim=2)
    predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

    # Merge I-<Tag> with <Tag>
    #Need to implement: checking previous token was the same tag, majority voting, ##
    merged_tokens = [] # list of [tag, token sequence]
    prev_tag, prev_token = "", ""
    current_tag = None

    # print(list(zip(decoded, predicted_token_class)))
    for token, tag in zip(decoded, predicted_token_class):
      # exclude CLS and SEP
      if token in ['<s>', '</s>']:
          continue

      if not token.startswith("Ġ"): #existing token token
          if len(merged_tokens) > 0:
              merged_tokens[-1][1] += token #add to previous word
              continue  # doing no majority voting, go to the next word
          else:
              token = token[1:]  # first observed token, act as if this subword is a complete word

      if tag.startswith('I-'):
          tag = tag[2:]
          # Check if the current "I-" tag matches the previous entry's tag
          if current_tag is not None and tag == current_tag:
              merged_tokens[-1][1] += ' ' + token
          else:
              merged_tokens.append([tag, token])
          current_tag = tag
          continue

      current_tag = tag
      merged_tokens.append([current_tag, token])

    # print(merged_tokens)
    # return merged_tokens
    return case_outputs(merged_tokens, title) # returns cased and most similar outputs
    # find a way to avoid casing?

def get_most_similar_word(text, word):
    #problem: EUR 42,5 when word = "eur 42 , 5"

    if " " in word:
        #check1: try to see by subtoken
        subtoken1 = word.split(" ")[0] #first subtoken in sequence
        start_index = text.find(get_most_similar_word(text, subtoken1))
        subtoken2 = word.split(" ")[-1] #last subtoken
        end_sim_word = get_most_similar_word(text, subtoken2)
        end_index = text.find(end_sim_word) + len(end_sim_word) #go till the end
        if start_index != -1 and start_index < end_index: #if not, just get most similar word
            return text[start_index:end_index]

    #check2: check a space-separated word
    words = pd.Series(text.split(" "))
    scores1 = words.apply(lambda x: similarity_score(x, word)) #score if lowercase word
    scores2 = words.apply(lambda x: similarity_score(x, word.upper())) #score if uppercase word
    scores3 = words.apply(lambda x: similarity_score(x, word[0].upper() + word[1:]))
    scores = pd.concat([scores1, scores2],axis=1).max(axis=1)
    return words[scores.argmax()]

def similarity_score(word1, word2):
    # Calculate the Levenshtein distance
    levenshtein_distance = distance.levenshtein(word1, word2)

    # Calculate the similarity score (normalized)
    max_length = max(len(word1), len(word2))
    similarity = 1 - (levenshtein_distance / max_length)

    return similarity

def case_outputs(outputs, text):
    lower_text = text.lower()

    for i in range(len(outputs)):
        token = outputs[i][1]
        subtoken = token.split(" ")[0] #consider only the first item in a multi-token
        start_index = lower_text.find(subtoken)
        last_subtoken = token.split(" ")[-1]
        end_index = lower_text.find(last_subtoken) + len(last_subtoken)
        cased_token = text[start_index:end_index]


        if start_index == -1 or start_index > end_index or cased_token == "": #not found
            outputs[i][1] = get_most_similar_word(text, token)
            continue

        outputs[i][1] = cased_token

    return outputs

def train_predict_and_write(bs, lr):
    training_args = TrainingArguments(
        output_dir="roberta_ebay_model",
        learning_rate=lr,
        per_device_train_batch_size=bs, #16
        per_device_eval_batch_size=bs, #16
        num_train_epochs=15,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_ebay_train,
        eval_dataset=tokenized_ebay_valid,
        tokenizer=lowercased_tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    result_list = []
    count = 0
    for index, row in test_records.iterrows():
        if count % 5000 == 0: #5000
          print(count)
        record_number = row['Record Number']
        title = row['Title']
        tagged_tokens = process_title(title, gpu=True)

        for tag, token_sequence in tagged_tokens:
            result_list.append({
                'Record Number': record_number,
                'Aspect Name': tag,
                'Aspect Value': token_sequence
            })
        count += 1

    result = pd.DataFrame(result_list)
    result = result.loc[(result["Aspect Name"] != "No Tag")&(result["Aspect Name"] != "Obscure"), :] # remove No Tag, obscure
    result["Record Number"] = result["Record Number"].astype(int)
    out = result.loc[result["Aspect Value"]!="null", :]

    filename = f"results_1126_roberta_lr{lr}_bs{bs}.tsv"
    out.to_csv(data_drive + filename, sep="\t", na_rep = "null", header=False, index=False)
    print(f"wrote to {filename}")

for batch_size in batch_sizes:
  for learning_rate in learning_rates:
    train_predict_and_write(batch_size, learning_rate)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.726575,0.762016,0.710217,0.735206,0.793408
2,1.010800,0.552055,0.829152,0.797007,0.812762,0.853656
3,1.010800,0.469714,0.863815,0.845858,0.854742,0.884726
4,0.333200,0.427067,0.880837,0.870492,0.875634,0.899316
5,0.333200,0.433297,0.882537,0.877767,0.880146,0.902828
6,0.162800,0.433373,0.891387,0.877767,0.884525,0.908231
7,0.162800,0.467946,0.890193,0.883068,0.886616,0.909042
8,0.089400,0.472577,0.891838,0.890448,0.891143,0.911383
9,0.089400,0.476473,0.897008,0.894398,0.895701,0.915616
10,0.047900,0.477951,0.897246,0.893982,0.895611,0.915706


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


0
5000
10000
15000
20000
wrote to results_1126_roberta_lr0.0001_bs16.tsv


In [ ]:
# writing my own training loop


<Tip>

For a more in-depth example of how to finetune a model for token classification, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb)
or [TensorFlow notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification-tf.ipynb).

</Tip>

In [ ]:
torch.save(model.state_dict(), data_drive+"/models/roberta_model_lower_large")

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
pass

In [ ]:
# from tqdm.auto import tqdm

# progress_bar = tqdm(range(num_training_steps))

# model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)

  0%|          | 0/1500 [00:00<?, ?it/s]

RuntimeError: ignored

## Load model (skip)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    'numind/entity-recognition-general-sota-v1',
    output_hidden_states=True, num_labels=70, id2label=id2label, label2id=label2id
)
load_name = "/models/roberta_model_lower_large"
# model.load_state_dict(torch.load(data_drive + load_name, map_location=torch.device('cpu'))) #if CPU
model.load_state_dict(torch.load(data_drive + load_name)) #if GPU


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at numind/entity-recognition-general-sota-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

## Inference on a single sample (test)

In [ ]:
#ideas for improvement - some kind of Autoregression; previous token predicted class impacts next token?
# all of this data is for shoes.

In [ ]:
model.eval()
model.to("cuda") #UNCOMMENT FOR GPU
pass

In [ ]:
sample = test_records.loc[test_records["Record Number"] == "6454", "Title"].values[0]
sample

'Minu Damen Schuhe Schnürschuhe 207536 Leder schwarz EUR 42'

## Case Outputs Functionality

In [ ]:
def similarity_score(word1, word2):
    # Calculate the Levenshtein distance
    levenshtein_distance = distance.levenshtein(word1, word2)

    # Calculate the similarity score (normalized)
    max_length = max(len(word1), len(word2))
    similarity = 1 - (levenshtein_distance / max_length)

    return similarity

In [ ]:
def get_most_similar_word(text, word):
    #problem: EUR 42,5 when word = "eur 42 , 5"

    if " " in word:
        #check1: try to see by subtoken
        subtoken1 = word.split(" ")[0] #first subtoken in sequence
        start_index = text.find(get_most_similar_word(text, subtoken1))
        subtoken2 = word.split(" ")[-1] #last subtoken
        end_sim_word = get_most_similar_word(text, subtoken2)
        end_index = text.find(end_sim_word) + len(end_sim_word) #go till the end
        if start_index != -1 and start_index < end_index: #if not, just get most similar word
            return text[start_index:end_index]

    #check2: check a space-separated word
    words = pd.Series(text.split(" "))
    scores1 = words.apply(lambda x: similarity_score(x, word)) #score if lowercase word
    scores2 = words.apply(lambda x: similarity_score(x, word.upper())) #score if uppercase word
    scores3 = words.apply(lambda x: similarity_score(x, word[0].upper() + word[1:]))
    scores = pd.concat([scores1, scores2],axis=1).max(axis=1)
    return words[scores.argmax()]

In [ ]:
def case_outputs(outputs, text):
    lower_text = text.lower()

    for i in range(len(outputs)):
        token = outputs[i][1]
        subtoken = token.split(" ")[0] #consider only the first item in a multi-token
        start_index = lower_text.find(subtoken)
        last_subtoken = token.split(" ")[-1]
        end_index = lower_text.find(last_subtoken) + len(last_subtoken)
        cased_token = text[start_index:end_index]


        if start_index == -1 or start_index > end_index or cased_token == "": #not found
            outputs[i][1] = get_most_similar_word(text, token)
            continue

        outputs[i][1] = cased_token

    return outputs

In [ ]:
#function to obtain tag predictions and merge aspect values
def process_title(title, gpu=True):
    inputs = lowercased_tokenizer(title.lower(), return_tensors="pt")
    if gpu:
        cuda_in = inputs.input_ids.to("cuda")
    else:
        cuda_in = inputs.input_ids
    decoded = lowercased_tokenizer.convert_ids_to_tokens(cuda_in[0])

    model.eval()
    with torch.no_grad():
        logits = model(cuda_in).logits
    predictions = torch.argmax(logits, dim=2)
    predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

    # Merge I-<Tag> with <Tag>
    #Need to implement: checking previous token was the same tag, majority voting, ##
    merged_tokens = [] # list of [tag, token sequence]
    prev_tag, prev_token = "", ""
    current_tag = None

    # print(list(zip(decoded, predicted_token_class)))
    for token, tag in zip(decoded, predicted_token_class):
      # exclude CLS and SEP
      if token in ['<s>', '</s>']:
          continue

      if not token.startswith("Ġ"): #existing token token
          if len(merged_tokens) > 0:
              merged_tokens[-1][1] += token #add to previous word
              continue  # doing no majority voting, go to the next word
          else:
              token = token[1:]  # first observed token, act as if this subword is a complete word

      if tag.startswith('I-'):
          tag = tag[2:]
          # Check if the current "I-" tag matches the previous entry's tag
          if current_tag is not None and tag == current_tag:
              merged_tokens[-1][1] += ' ' + token
          else:
              merged_tokens.append([tag, token])
          current_tag = tag
          continue

      current_tag = tag
      merged_tokens.append([current_tag, token])

    # print(merged_tokens)
    # return merged_tokens
    return case_outputs(merged_tokens, title) # returns cased and most similar outputs
    # find a way to avoid casing?


In [ ]:
sample

'Minu Damen Schuhe Schnürschuhe 207536 Leder schwarz EUR 42'

In [ ]:
process_title(sample, gpu=True)

[('<s>', 'Obscure'), ('Ġmin', 'Marke'), ('u', 'I-Marke'), ('Ġdam', 'Abteilung'), ('en', 'Abteilung'), ('Ġsch', 'Produktart'), ('u', 'Produktart'), ('he', 'I-Produktart'), ('Ġs', 'I-Produktart'), ('chn', 'I-Produktart'), ('Ã¼', 'I-Produktart'), ('rs', 'I-Produktart'), ('chu', 'I-Produktart'), ('he', 'I-Produktart'), ('Ġ20', 'Herstellernummer'), ('75', 'Herstellernummer'), ('36', 'I-Herstellernummer'), ('Ġled', 'Obermaterial'), ('er', 'Obermaterial'), ('Ġsch', 'Farbe'), ('war', 'Farbe'), ('z', 'I-Farbe'), ('Ġe', 'EU-Schuhgröße'), ('ur', 'I-EU-Schuhgröße'), ('Ġ42', 'I-EU-Schuhgröße'), ('</s>', 'Obscure')]
[['Marke', 'Ġminu'], ['Abteilung', 'Ġdamen'], ['Produktart', 'Ġschuhe ĠschnÃ¼rschuhe'], ['Herstellernummer', 'Ġ207536'], ['Obermaterial', 'Ġleder'], ['Farbe', 'Ġschwarz'], ['EU-Schuhgröße', 'Ġeur Ġ42']]


[['Marke', 'Minu'],
 ['Abteilung', 'Damen'],
 ['Produktart', 'Schuhe Schnürschuhe'],
 ['Herstellernummer', '207536'],
 ['Obermaterial', 'Leder'],
 ['Farbe', 'schwarz'],
 ['EU-Schuhgröße', 'EUR 42']]

## Do prediction on test dataset


In [ ]:
result_list = []
count = 0
for index, row in test_records.iterrows():
    if count % 1000 == 0: #1000
      print(count)
    record_number = row['Record Number']
    title = row['Title']
    tagged_tokens = process_title(title, gpu=True)

    for tag, token_sequence in tagged_tokens:
        result_list.append({
            'Record Number': record_number,
            'Aspect Name': tag,
            'Aspect Value': token_sequence
        })
    count += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


In [ ]:
result = pd.DataFrame(result_list)
result = result.loc[(result["Aspect Name"] != "No Tag")&(result["Aspect Name"] != "Obscure"), :] # remove No Tag, obscure
result["Record Number"] = result["Record Number"].astype(int)

In [ ]:
result.tail()

,Record Number,Aspect Name,Aspect Value
227993,30000,Marke,Puma
227994,30000,Produktart,Turnschuhe
227995,30000,Farbe,weiß
227996,30000,Abteilung,unisex
227997,30000,EU-Schuhgröße,43


In [ ]:
test_records.loc[test_records["Record Number"] == "10030", "Title"].values #check Women's --> i'm pretty sure it breaks

array(['Adidas Super Light Loop Runner Black White Mens Trainers - C77018'],
      dtype=object)

In [ ]:
result.loc[result["Record Number"] == 10030,:]

,Record Number,Aspect Name,Aspect Value
45858,10030,Marke,Adidas
45859,10030,Produktlinie,Super
45860,10030,Modell,Light Loop
45861,10030,Produktart,Runner
45862,10030,Farbe,Black White
45863,10030,Abteilung,Mens
45864,10030,Produktart,Trainers
45866,10030,Herstellernummer,C77018


In [ ]:
# check to make sure no NaN values
np.sum(result.isna())

Record Number    0
Aspect Name      0
Aspect Value     0
dtype: int64

In [ ]:
result.loc[(result["Aspect Name"]=="No Tag")|(result["Aspect Name"]=="Obscure"), :]

,Record Number,Aspect Name,Aspect Value


In [ ]:
result.loc[result["Aspect Value"]=="null", :] #still values to fix, but let's test for now
# CHECK NA REP: 11233, 11817

,Record Number,Aspect Name,Aspect Value


In [ ]:
out = result.loc[result["Aspect Value"]!="null", :]
# out = out.loc[out["Aspect Value"] != "Obscure", :]
out

,Record Number,Aspect Name,Aspect Value
0,5001,Marke,NIKE
1,5001,Produktlinie,FREE
2,5001,Modell,RUN
3,5001,Modell,3 SHIELD 5.0
4,5001,Stil,SNEAKERS
...,...,...,...
227993,30000,Marke,Puma
227994,30000,Produktart,Turnschuhe
227995,30000,Farbe,weiß
227996,30000,Abteilung,unisex


In [ ]:
# save to file
filename = "results_1124_roberta_large_lower_results.tsv"
out.to_csv(data_drive + "/results/" + filename, sep="\t", na_rep = "null", header=False, index=False)

## Tests (skip)

In [ ]:
#testing the I-tag fix

text = train_df.Title[0]
inputs = tokenizer(text, return_tensors="pt")
cuda_in = inputs.input_ids.to("cuda")
decoded = tokenizer.convert_ids_to_tokens(cuda_in[0])
model.eval() #not sure if this is correct
with torch.no_grad():
    logits = model(cuda_in).logits
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
dict(zip(decoded, predicted_token_class))


merged_tokens = [] # list of [tag, token sequence]
prev_tag, prev_token = "", ""
current_tag = None

for token, tag in zip(decoded, predicted_token_class):
    if token.startswith("##"):
        if len(merged_tokens) > 0:
            merged_tokens[-1][1] += token[2:]
            continue  # doing no majority voting, go to the next word
        else:
            token = token[2:]  # first observed token, act as if this subword is a complete word

    if tag.startswith('I-'):
        tag = tag[2:]
        # Check if the current "I-" tag matches the previous entry's tag
        if current_tag is not None and tag == current_tag:
            merged_tokens[-1][1] += ' ' + token
        else:
            merged_tokens.append([tag, token])
        current_tag = tag
        continue

    if tag != 'O' and token not in ['[CLS]', '[SEP]']:
        current_tag = tag
        merged_tokens.append([current_tag, token])
    else:
        current_tag = None

print(merged_tokens)


In [ ]:
ex_title = train_df.loc[0, "Title"]
ex_title

In [ ]:
tokenizer(ex_title)

In [ ]:
merged_tokens = [] # list of [tag, token sequence]
prev_tag, prev_token = "", ""
current_tag = None

for token, tag in zip(decoded, predicted_token_class):
    if token.startswith("##"):
        if len(merged_tokens) > 0:
            merged_tokens[-1][1] += token[2:]
            continue  # doing no majority voting, go to the next word
        else:
            token = token[2:]  # first observed token, act as if this subword is a complete word

    if tag.startswith('I-'):
        tag = tag[2:]
        # Check if the current "I-" tag matches the previous entry's tag
        if current_tag is not None and tag == current_tag:
            merged_tokens[-1][1] += ' ' + token
        else:
            merged_tokens.append([tag, token])
        current_tag = tag
        continue

    if tag != 'O' and token not in ['[CLS]', '[SEP]']:
        current_tag = tag
        merged_tokens.append([current_tag, token])
    else:
        current_tag = None

print(merged_tokens)
